In [2]:
import simpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pandas as pd
import time
from scipy import stats
import heapq


Queuing theory tells us that for FIFO scheduling the average waiting times are shorter for an
$M/M/n$ queue and a system load $\rho$ and processor capacity $\mu$ than for a single $M/M/1$ queue
with the same load characteristics (and thus an n-fold lower arrival rate). Of course, $\rho$ must be
less than one, but the experiment only becomes interesting when $\rho$ is not much less than one.

$\lambda$ then becomes $\lambda = \rho*n*\mu$.

Code sources: "https://simpy.readthedocs.io/en/latest/examples/bank_renege.html" and "https://www.youtube.com/watch?v=eSNfC-HOl44"

In [13]:
# M/M/1 (M/M/n) queueing system
def generate_interarrival(mu,n,rho):
    return random.expovariate(mu*n*rho)


def source(env, n, rho, mu, servers):
    """Source generates customers randomly based on exponential distribution"""
    i = 0
    while True:
        i += 1
        c = customer(env, i, servers, mu)
        env.process(c)
        t = random.expovariate(mu*n*rho)
        yield env.timeout(t)   

total_waiting_time = []

def customer(env, customer, servers, mu):
    """This generator function models the behavior of a customer in the queue."""
    time_of_arrival = env.now
    len(servers.queue)
    with servers.request() as req:    
        #print(env.now, 'customer {} arrives'.format(customer))
        yield req
        time_of_handling = random.expovariate(mu)
        #print(env.now, 'customer {} is being served'.format(customer))
        yield env.timeout(time_of_handling)
        waiting_time = env.now
        #print(env.now, 'customer {} departs'.format(customer))
        total_waiting_time.append(waiting_time-time_of_arrival)

### Setup and start the simulation

# Initial values            
mu = 1  # Parameter mu
rho = 0.9 #[0.7,0.85,0.9,0.95,0.99] # Parameter rho (varying for values close to 1)
n = 2 # Number of servers, in our case 1, 2 and 4

env = simpy.Environment()

# Start processes and run
servers = simpy.Resource(env, capacity=n)
env.process(source(env, n, rho, mu, servers))
env.run(until=200)
print(f"Total customers served: {len(total_waiting_time)}")
print(f"Average waiting time: {sum(total_waiting_time) / len(total_waiting_time) if total_waiting_time else 0}")

Total customers served: 362
Average waiting time: 5.506389280338976


In [6]:
# Defining function for simulating M/M/n queue system

# Setup and start the simulation
def sim_run(n,rho,mu,sim_duration):
    env = simpy.Environment()
    # Start processes and run
    servers = simpy.Resource(env, capacity=n)
    env.process(source(env, n, rho, mu, servers))
    env.run(until=sim_duration)
    
# Looping over the simulation and extracting mean and standard deviation
# Initial values
sim_duration = 500     
mu = 1  # Parameter mu
rho_vals = [0.9,0.95] #[0.7,0.85,0.92,0.95,0.99]
n = [1] #[1,2,4]

# Running simulation to calculate mean
sim_runs = 20
data = []

for n_servers in n:
    for rho in rho_vals:
        for i in range(sim_runs):
            total_waiting_time = []
            sim_run(n=n_servers, rho=rho, mu=mu, sim_duration=sim_duration)
            averages = np.concatenate((np.arange(100, 1000, 100), np.arange(1000, 10000, 1000), np.arange(10000, 100001, 10000)))
            df = pd.DataFrame({
                'run': [i] * len(averages),
                'rho': [rho] * len(averages),
                'number of servers': [n_servers] * len(averages),
                'number of customers': averages,
                'average waiting time': [np.mean(total_waiting_time[:n_customers]) for n_customers in averages]
            })
            data.append(df)

        print(f'rho={rho} for server number {n_servers} finished')

# Concatenate the list of DataFrames
result_df = pd.concat(data, ignore_index=True)    

rho=0.9 for server number 1 finished
rho=0.95 for server number 1 finished


In [ ]:
# Plot the mean and standard deviation of the waiting time

SJF implementation

#1 Simple SJF

In [12]:

def source(env, mu, n, rho, queue):
    while True:
        time_of_handling = random.expovariate(mu)
        arrival_time = env.now
        heapq.heappush(queue, (time_of_handling, arrival_time))
        #print(f"Bisy: {servers.count}")
        yield env.timeout(random.expovariate(mu*n*rho))

def do_work(env, client, servers, time_to_finish, total_waiting_time):
    with servers.request() as req:
        yield req
        #print(f"Number of servers in util: {servers.count}")
        yield env.timeout(time_to_finish)
        total_waiting_time.append(env.now - client)

def customer_initialise(env, servers, queue, total_waiting_time):
    while True:
        if queue:
            time_to_finish, time_arival = heapq.heappop(queue)
            env.process(do_work(env, time_arival, servers, time_to_finish, total_waiting_time))
        else:
            yield env.timeout(0.0001)

mu = 1.0          
n = 2             
rho = 0.95         
sim_time = 200    

env = simpy.Environment()
servers = simpy.Resource(env,capacity=n)
priority_queue = []
total_waiting_time = []

env.process(source(env, mu, n, rho, priority_queue))
env.process(customer_initialise(env, servers, priority_queue, total_waiting_time))
env.run(until=sim_time)

print(f"Total customers served: {len(total_waiting_time)}")
print(f"Average waiting time: {sum(total_waiting_time) / len(total_waiting_time) if total_waiting_time else 0}")

Total customers served: 364
Average waiting time: 3.908082198304474


SJF with halting the precesses if faster customer arrives

In [11]:

class Client:
    def __init__(self, time_of_arrival, time_of_handling):
        self.time_of_arrival = time_of_arrival
        self.time_of_handling = time_of_handling

def source(env, mu, n, rho, queue):
    while True:
        time_of_handling = random.expovariate(mu)
        arrival_time = env.now
        heapq.heappush(queue, (time_of_handling, Client(arrival_time, time_of_handling)))
        yield env.timeout(random.expovariate(mu*n*rho))

def customer_service(env, client, servers, time_to_finish, total_waiting_time, queue):
    with servers.request() as req:
        yield req
        if time_to_finish <= 0.01:
            yield env.timeout(time_to_finish)
            total_waiting_time.append(env.now - client.time_of_arrival)
        else:
            yield env.timeout(0.01)
            arrival_time = client.time_of_arrival
            time_of_handling = client.time_of_handling - 0.01
            heapq.heappush(queue, (time_of_handling, Client(arrival_time, time_of_handling)))
            
def customer_proceed(env, servers, queue, total_waiting_time):
    while True:
        if queue:
            time_to_finish, client = heapq.heappop(queue)
            env.process(customer_service(env, client, servers, time_to_finish, total_waiting_time, queue))
        else:
            yield env.timeout(0.0001)

mu = 1.0         
n = 2             
rho = 0.95         
sim_time = 200   

env = simpy.Environment()

servers = simpy.Resource(env,capacity=n)
priority_queue = []
total_waiting_time = []

env.process(source(env, mu, n, rho, priority_queue))
env.process(customer_proceed(env, servers, priority_queue, total_waiting_time))
env.run(until=sim_time)

print(f"Total customers served: {len(total_waiting_time)}")
print(f"Average waiting time: {sum(total_waiting_time) / len(total_waiting_time) if total_waiting_time else 0}")

Total customers served: 365
Average waiting time: 2.824298002217367
